# Interpretability of a Convolutional Neural Network

**Notebook created by [Daniel Fojo](https://www.linkedin.com/in/daniel-fojo/) and [Xavier Giro-i-Nieto](https://imatge.upc.edu/web/people/xavier-giro) for the [Postgraduate course in artificial intelligence with deep learning](https://www.talent.upc.edu/ing/estudis/formacio/curs/310400/postgrau-artificial-intelligence-deep-learning/) in [UPC School](https://www.talent.upc.edu/ing/) (2019).**

**Based on previous versions by [Amaia Salvador](https://www.linkedin.com/in/amaiasalvador/) ([Persontyle](https://github.com/telecombcn-dl/2017-persontyle) 2017) and [Daniel Fojo](https://www.linkedin.com/in/daniel-fojo/) ([Barcelona Technology School](https://barcelonatechnologyschool.com/) 2019)**

**Related [slides](https://github.com/telecombcn-dl/dlai-2019/raw/master/slides/dlai_2019_d07l1_interpretability.pdf) by [Xavier Giro-i-Nieto](https://imatge.upc.edu/web/people/xavier-giro) from [Deep Learning for Artificial Intelligence](https://telecombcn-dl.github.io/dlai-2019/) (UPC TelecomBCN 2019)**

**Related video lectures from [Amaia Salvador](https://www.youtube.com/watch?v=YQvTxkPV8LQ&feature=emb_logo) and [Eva Mohedano](https://www.youtube.com/watch?v=SsHohytl1NA&feature=emb_logo) from Deep Learning for Computer Vision ([UPC TelecomBCN 2016](http://imatge-upc.github.io/telecombcn-2016-dlcv/)) and ([UPC TelecomBCN 2018](https://telecombcn-dl.github.io/2018-dlcv/)), respectively.**

![Daniel Fojo](https://telecombcn-dl.github.io/2018-dlcv/img/assistants/DanielFojo-160x160.jpg)
![Amaia Salvador](https://telecombcn-dl.github.io/2017-dlcv/img/instructors/AmaiaSalvador.jpg)
![Eva Mohedano](https://telecombcn-dl.github.io/2018-dlcv/img/instructors/EvaMohedano.jpg)
![Xavier Giro-i-Nieto](https://telecombcn-dl.github.io/dlai-2019/img/instructors/XavierGiro.jpg)

So far we have learned how to train models for image classification, and we have evaluated their performance in terms of accuracy. But, what did these models learn? Let's try to understand that.

## Filters

We will first train a simple model on CIFAR10, and then we will try to visualize how it works.

In [0]:
import copy
import time
import itertools

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import matplotlib.pyplot as plt  
%matplotlib inline

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


if not torch.cuda.is_available():
    raise Exception("You should enable GPU in the runtime menu.")
device = torch.device("cuda:0")

Load the data.

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)


Declare the model.
Related docs: [torch.nn.Conv2D](https://pytorch.org/docs/stable/nn.html#conv2d)

In [0]:
model = nn.Sequential(
        nn.Conv2d(3, 32, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 32, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.5),
        nn.Conv2d(32, 64, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 64, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.5),
        nn.Flatten(),
        nn.Linear(8 * 8 * 64, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 10),
)
model.to(device)

**Exercise 1:**
How many convolutional filters are there in the first layer ?

Declare loss function and optimizer.

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Train the model.

In [0]:
t = time.time()
model.train()
epochs = 5
for epoch in range(epochs):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader):
        # get the inputs to gpu; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        if i % 300 == 299:    # print every 300 mini-batches
            print(f"Epoch {epoch+1}/{epochs} [{i+1}/{len(trainloader)}] loss: {loss.item():.2f}")

print('Finished Training')
print(f"Time: {(time.time() - t):.1f}s")

Disable gradient computation for the rest of the session, we will not need it.

In [0]:
# Disable gradients computation
torch.set_grad_enabled(False);

Evaluate our model.

In [0]:
correct = 0
total = 0
model.eval()
for data in testloader:
    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {correct / total*100:.1f}%")

The following function `display_filters` allows visualizing the convolutional filters in a layer. If the filter has more than 3 channels, it will only consider these ones in the visualization.

In [0]:
def display_filters(weights):
    N = int(np.ceil(np.sqrt(weights.shape[0])))
    f, axarr = plt.subplots(N, N)
    scaled = (weights-weights.min())/(weights.max()-weights.min()) # Scale the weights for better plotting

    p = 0
    for i in range(N):
        for j in range(N):
            # empty plot white when out of kernels to display
            if p >= scaled.shape[0]:
                krnl = torch.ones((scaled.shape[2], scaled.shape[3], 3))
            else:
                if scaled.shape[1] == 1: 
                    krnl = scaled[p, :, :, :].permute(1, 2, 0)
                    axarr[i, j].imshow(krnl)
                # NEW: If we have 3 channels plot with color
                elif scaled.shape[1] == 3: 
                    krnl = scaled[p, :, :, :].permute(1, 2, 0)
                    axarr[i, j].imshow(krnl)
                else: # If we don't have either 1 or 3 channels, just plot the first channel
                    krnl = scaled[p,0,:,:]
                    axarr[i, j].imshow(krnl, cmap='gray')
            axarr[i, j].axis('off')
            p += 1

Let's exploit the function `display_filters` to understand better the convolutional neural network that we have trained:

In [0]:
# Display the architecture of the model
model

We can get the values of the weights for the first the kernels of the first layer and visualize them, since they have 3 channels.

In [0]:
# get the weights from the first layer to cpu
weights = model[0].weight.cpu().detach() # Detach, since we do not need the gradients
weights.shape
display_filters(weights)
plt.show()


Do these visualizations help you understand what the model learned? If not, later you can try to visualize the activations when these filters are convolved with the image. For now, let's move on to other visualization methods, but you can come back to this later.

**Exercise 2:** Try to visualize filters in the second convolutional layer.

In [0]:
# TODO
weights=TODO
display_filters(weights)

**Exercise 3:** Why are the visualizations of the convolutional filters of the first layer colored, but the ones from the second layer plotted in gray-scale ?

## Activations

We can also visualize the model's activations to our data samples to understand what the model learned.

We will create a model that we will use as feature extractor. For that, we need to pick the layer that we want to use.

**Exercise 4:** Since our NN model is of the type `nn.Sequential`, we can easily define a new one with any number of sequential layers just using a *slice*. Take a look at the layer numbers, and declare a new model names `extractor` that should have all the layers of `model` until and including the first `Linear` layer. Remember that Python slices do not include the last number (e.g. 0:5 = [0, 1, 2, 3, 4]).

In [0]:
# TODO
extractor=model[TODO]

# Visualize the resulting layer after slicing
extractor

Once we have our new neural network `extractor`, we can load the data and forward it through the network to get the activations:

In [0]:
feats = []
for data in testloader:
    inputs, _ = data[0].to(device), data[1].to(device)
    feats.append(extractor(inputs).cpu().numpy())

feats = np.concatenate(feats)

# Check that the extracted features correspond to the 10000 test images and the
# amount of neurons in the last considered fully connected layer
feats.shape

Once we have extracted activations for all samples in our test set, we will use different visualization tools to understand what the model learned.

### Finding per unit top K samples

Let's now find the K images with highest activation for each neuron in the layer, using the original extracted activations:

In [0]:
testimages = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=None)

K = 10
idxs_top10 = np.argsort(feats, axis=0)[::-1][0:K, :]
picked_samples = np.zeros((K, 128, 32, 32, 3), dtype=float)
for i in range(idxs_top10.shape[0]):
    for j in range(idxs_top10.shape[1]):
        picked_samples[i, j, :, :, :] = np.asarray(testimages[idxs_top10[i, j]][0])/255

In [0]:
picked_samples.shape
# The shape of the tensor corresponds to: 
# (n_images,n_units,nb_rows,nb_cols,nb_channels)

```picked_samples``` now contains the 10 images with highest activation for each neuron. Let's visualize these images for some neurons:

**Exercise 5:** The following array defines which units are selected to display their top K images. However, the visualization will crash because one of the values is wrong. Find it and correct it.

In [0]:
units = [1, 2, 4, 14, 23, 29, 128] 

In [0]:
nunits = len(units)
ims = picked_samples[:, units, :, :].squeeze()

def vis_topk(ims,units):
    f, axarr = plt.subplots(ims.shape[0],ims.shape[1],figsize=(10,10))
    
    for i in range(ims.shape[0]):
        for j in range(ims.shape[1]):

            axarr[i,j].imshow(ims[i,j,:,:,:])
            axarr[i,j].axis('off')
            axarr[0,j].set_title('unit '+ str(units[j]))
            
vis_topk(ims, units)
ims.shape
#(n_ims,n_units_picked,nb_rows,nb_cols,nb_channels)

Do all units respond to distinguishable concepts? Are there units that respond to similar things? Did you find any units with semantic meaning? You can try for different units and see what images they like the most.

### Occlusion experiment

In this section, we will find which parts of the image contribute to the activation the most. We will create a small NxN occluder patch and slide it through each image with a stride of 2, and then feed each occluded image through the network. Later, we will compute the difference between the activations between the original image and the occluded ones, and create a difference map that we will overlay over the image for visualization. 

In [0]:
def occ_images(ims, occ=(11, 11), stride=4):
    
    # Reshape to put top images for all units stacked together
    ims = np.rollaxis(ims, 1, 0)
    ims = np.reshape(ims, (ims.shape[0]*ims.shape[1], ims.shape[2], ims.shape[3], ims.shape[4]))
    ims_acc = ims
    
    # slide 
    npos = 1
    st = int(np.floor(occ[0]/2))
    
    # slide occluder, set pixels to 0 and stack to matrix
    for i in range(st, ims.shape[1], stride):
        for j in range(st, ims.shape[2], stride):
            ims_occ = copy.deepcopy(ims)
            ims_occ[:, i-st:i+occ[0]-st, j-st:j+occ[1]-st, :] = 0
            ims_acc = np.vstack((ims_acc, ims_occ))
            npos += 1
            
    return ims_acc

# Feed the previously obtained top K images through the occluding function
ims_acc = occ_images(ims)
print(ims_acc.shape)

Let's visualize some of the images with the occluded region in different positions:

In [0]:
f, axarr = plt.subplots(10,ims.shape[1],figsize=(10,10))
ims_acc_r = ims_acc.reshape(ims_acc.shape[0]//(ims.shape[0]*ims.shape[1]),
                                ims.shape[1], ims.shape[0],
                                ims_acc.shape[1], ims_acc.shape[2], ims_acc.shape[3])
for i in range(10):
    for j in range(ims.shape[1]):
        axarr[i,j].imshow(ims_acc_r[2*i,j,0,:,:,:])
        axarr[i,j].axis('off')
plt.show()

We should pick an occluder that is large enough to cover significant parts of objects. 11x11 is the default one, but you can experiment with other sizes and see their effect.

```ims_acc``` contains all images with the occluder set at different positions. Let's run these through our extractor:

In [0]:
# Build a PyTorch tensor from the occluded images
ims_acc_tensor = torch.tensor(ims_acc, dtype=torch.float).permute(0, 3, 1, 2).to(device)

 # Normalize data
ims_acc_tensor = (ims_acc_tensor-0.5)/0.5 

# Feed the occluded imaged through the network
output = extractor(ims_acc_tensor)

# Transform the output tensor to numpy
feats_occ = output.cpu().numpy()
feats_occ.shape

Now that we have the features, we can compute the difference between the original activation and the activation for each of the occluded images:

In [0]:
feats_r = np.reshape(feats_occ,(feats_occ.shape[0] // (ims.shape[0] * ims.shape[1]),
                                ims.shape[1], ims.shape[0], feats_occ.shape[1]))

distances = feats_r[0] - feats_r[1:] # original activation minus all the occluded ones
distances = np.rollaxis(distances, 0, 4)

Reshaping the distance array into a 2D map will give a mask that we can display on top of the images:

In [0]:
s = int(np.sqrt(distances.shape[3]))

heatmaps = np.zeros((distances.shape[0],distances.shape[1],distances.shape[3]))
for i in range(len(units)):    
    heatmaps[i] = distances[i,:,units[i],:]
heatmaps.shape

Let's declare and run a function to display the masks on top of the images:

In [0]:
def vis_occ(ims,heatmaps,units,th=0.5,sig=2):
    
    from scipy.ndimage.interpolation import zoom
    from scipy.ndimage.filters import gaussian_filter
    import copy
    
    ims = np.rollaxis(ims,1,0)
    
    s = int(np.sqrt(heatmaps.shape[2]))
    heatmaps = np.reshape(heatmaps,(heatmaps.shape[0],heatmaps.shape[1],s,s))
    
    f, axarr = plt.subplots(ims.shape[1],ims.shape[0],figsize=(10,10))
    
    for i in range(ims.shape[0]):
        for j in range(ims.shape[1]):
            
            im = copy.deepcopy(ims[i,j,:,:,:])
            mask = copy.deepcopy(heatmaps[i,j,:,:])
            
            # Normalize mask
            mask = (mask - np.min(mask))/(np.max(mask)-np.min(mask))
            # Resize to image size
            mask = zoom(mask,float(im.shape[0])/heatmaps.shape[-1],order=1)
            # Apply gaussian to smooth output
            mask = gaussian_filter(mask,sigma=sig)
            # threshold to obtain mask out of heatmap
            mask[mask>=th] = 1
            mask[mask<th] = 0.3
            
            # Mask all image channels
            for c in range(3):
                im[:,:,c] = im[:,:,c]*mask
                
            axarr[j,i].imshow(im)
            axarr[j,i].axis('off')
            axarr[0,i].set_title('unit '+ str(units[i]))


In [0]:
vis_occ(ims,heatmaps,units,th=0.5)

**Exercise 6**: The obtained masks are of course not perfect, but we get to see what parts of the image are most relevant for each unit in the layer. Are these masks what you expected? Do the picked neurons maximally respond to what you have previously guessed?

### Additional: t-SNE

Here we will display our learned features in a 2D space using t-SNE. To do this, we will use the provided function in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html). We will also reduce the dimensionality with PCA before running t-SNE to make it faster.

In [0]:
t = time.time()
# Reduce dimensionality with PCA before TSNE
n = 20
pca = PCA(n_components=n)
feats_nd = pca.fit_transform(feats)

# should do more iterations, but let's do the minimum due to time constraints
n_iter = 800
tsne = TSNE(n_components=2, random_state=0, n_iter=n_iter)
feats_2d = tsne.fit_transform(feats_nd)

print(f"Time: {(time.time() - t):.1f}s")
feats_2d.shape

Once we have our 2D features, we can display them with their class labels, to see if the learned features are discriminative enough.

In [0]:
cifar_labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
# 0: airplane
# 1: automobile
# 2: bird
# 3: cat
# 4: deer
# 5: dog
# 6: frog
# 7: horse
# 8: ship
# 9: truck
labels = [y for _, y in testimages]
s = 20 # area of samples. increase if you don't see cclear colors
plt.scatter(feats_2d[:,0], feats_2d[:,1], c=labels, cmap=plt.cm.get_cmap("jet", 10), s=20) # 10 because of the number of classes
plt.clim(-0.5, 9.5)
cbar = plt.colorbar(ticks=range(10))
cbar.ax.set_yticklabels(cifar_labels);

**Exercise 7:** What categories seem to be easier for our model? Which ones are confusing?